<a href="https://colab.research.google.com/github/lakshaygola/Anomaly-Detection/blob/main/Anomaly_Detetction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Preprocessing- scale the data
# Create the model - 1) Simple ANN 2)LSTM
# Train the model with train data
# Take the prediction and calculate the accuracy on test data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn.preprocessing as pre

In [3]:
# Loading the dataset
df = pd.read_csv('/content/drive/MyDrive/freq_encoded.csv')

In [4]:
df.head()

,Unnamed: 0,service,flag,src_bytes,dst_bytes,logged_in,count,srv_count,serror_rate,srv_serror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,outcome
0,0,62054,87459,181,5450,1,8,8,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0
1,1,62054,87459,239,486,1,8,8,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0
2,2,62054,87459,235,1337,1,8,8,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0
3,3,62054,87459,219,1337,1,6,6,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0
4,4,62054,87459,217,2032,1,6,6,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0


In [5]:
df.shape

(145586, 22)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145586 entries, 0 to 145585
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   145586 non-null  int64  
 1   service                      145586 non-null  int64  
 2   flag                         145586 non-null  int64  
 3   src_bytes                    145586 non-null  int64  
 4   dst_bytes                    145586 non-null  int64  
 5   logged_in                    145586 non-null  int64  
 6   count                        145586 non-null  int64  
 7   srv_count                    145586 non-null  int64  
 8   serror_rate                  145586 non-null  float64
 9   srv_serror_rate              145586 non-null  float64
 10  same_srv_rate                145586 non-null  float64
 11  diff_srv_rate                145586 non-null  float64
 12  srv_diff_host_rate           145586 non-null  float64
 13 

In [7]:
# Train test split 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [8]:
# X- all the features to train the model
# Y- target variable
Ydf = df['outcome']
Xdf = df.drop(['outcome'], axis = 1)

In [9]:
# Scaling the dataset 
scaler = MinMaxScaler(feature_range= (-1, 1))
X = scaler.fit_transform(Xdf)
# No Need for Y

In [10]:
len(df)

145586

In [11]:
# Train test split on the final data
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xdf, Ydf, test_size = 0.35, random_state = 30)

In [12]:
print(Xtrain.shape, Ytrain.shape)

(94630, 21) (94630,)


In [13]:
print(Xtest.shape, Ytest.shape)

(50956, 21) (50956,)


1) ANN (simple approch)

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# Creating the model (personal one)
model1 = Sequential()

model1.add(Dense(50, input_dim = 21, activation= 'LeakyReLU', kernel_initializer= 'he_uniform'))
# (Add Dropout) model1.add(Dropout(0.5))

model1.add(Dense(21, activation= 'LeakyReLU'))

# As we have to possible outcome for any scenario
model1.add(Dense(1, activation = 'sigmoid'))

model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model1.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 50)                1100      
_________________________________________________________________
dense_41 (Dense)             (None, 21)                1071      
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 22        
Total params: 2,193
Trainable params: 2,193
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Kaggle notebook model
model = Sequential()
    
model.add(Dense(21,input_dim =21,activation = 'relu',kernel_initializer='random_uniform'))
    
model.add(Dense(5, activation='relu'))
    
model.add(Dense(1, activation='sigmoid'))
    
model.compile(loss ='categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 21)                462       
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 110       
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 6         
Total params: 578
Trainable params: 578
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train on the train dataset
model1.fit(Xtrain, Ytrain, validation_data= (Xtest, Ytest), verbose= 2, epochs = 50, batch_size = 72)

In [ ]:
model2 = KerasClassifier(build_fn=create_ann,epochs=10,batch_size=64)

In [ ]:
model2.fit(Xtrain, Ytrain.values.ravel())

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
Ytest_pred1 = model1.predict(Xtest)
accuracy_score(Ytest, Ytest_pred1)

0.6047374205196641

In [15]:
from keras import initializers
from keras import optimizers

In [33]:
# Model 3 (using LSTM)
randomWt = initializers.RandomUniform(seed = 42)
optim = optimizers.Nadam(lr = 0.01, beta_1= 0.9, beta_2 = 0.8)
batch_size = 64

model3 = Sequential()

model3.add(LSTM(21, kernel_initializer = randomWt, batch_input_shape = ( Xtrain.shape[0], 1), return_sequences = True))
model3.add(Dropout(0.2))

model3.add(LSTM(10, kernel_initializer= randomWt, return_sequences= True, batch_input_shape= (Xtrain.shape[0], 1)))

model3.add(Dense(1, activation= 'sigmoid'))

model3.compile(loss = 'categorical_crossentropy', optimizer = optim, metrics = ['accuracy'])

model3.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


ValueError: ignored

In [31]:
model3.fit(Xtrain, Ytrain, epochs= 60, batch_size= 64, verbose=1,  validation_data=(Xtest, Ytest))

RuntimeError: ignored